In [1]:
'''
This ipynb file contains codes used in the ariticle Double coset operators and Eta-quotients
by Hai-Gang Zhou and Xiao-Jie Zhu in Journal of Number Theory.
It should run on SageMath 9.3 (or higher version) and Jupyter notebook.
Author: Xiao-Jie Zhu
e-mail: zhuxiaojiemath@outlook.com, mindkeep@live.cn
url: https://orcid.org/0000-0002-6733-0755
'''

import functools
import itertools
import time                   #for debuging and profiling
import shelve                 #for saving eta power series to a file

_debug_mode = False
def set_dbg(*mode):
    '''
    Direct calling set_dbg() change the debug mode.
    Calling with a boolean argument turn the debug mode to the given value.
    '''
    global _debug_mode
    if len(mode) == 0:
        _debug_mode = not _debug_mode
    else:
        _debug_mode = True if mode[0] else False
        

def eta_char_simple(A, n):
    '''
    Calculate the logarithm of the char. of
    Dedekind eta(n\tau) times 1/2pi*I at matrix A.
    '''
    (a, b), (c, d) = A[0], A[1]
    if c > 0:
        re = (a + d) / (24*c) * n + 1/2 * dedekind_sum(-d, c/n) - 1/8
    elif c < 0:
        re = (a + d) / (24*c) * n + 1/2 * dedekind_sum(d, -c/n) + 1/8
    elif c == 0 and a > 0:
        re = b / 24 * n
    else:
        re = -b / 24 * n - 1/4
    return re


def eta_char_simple_exp(A, n, b_algorithm_ds=True):
    '''
    Calculate the char. of Dedekind eta(n\tau) at matrix A
    using eta_char_simple if b_algorithm_ds=True or a formula of Petersson
    if b_algorithm_ds=False.
    Return an symbolic expression.
    '''
    if b_algorithm_ds:
        return exp(2 * sage.all.pi * sage.all.I * eta_char_simple(A, n))
    else:
        (a, b), (c, d) = A[0], A[1]
        temp = kronecker(d, abs(c)) if c % 2 == 1 else kronecker(c, d)
        if c % 2 == 1:
            expo = ((a+d-3)*c - b*d*(c^2-1)) / 24
        else:
            expo = ((a-2*d)*c - b*d*(c^2-1) + 3*d - 3) / 24
        return temp * exp(2 * sage.all.pi * sage.all.I * expo)
    
    
def _verify_eta_char_simple(n, A_entry_max=30):
    '''
    Use two algorithms to calculate char. of Dedekind eta(n\tau) at matrices
    with entries whose abs. values not exceeding A_entry_max.
    '''
    for a, b, c, d in itertools.product(srange(-A_entry_max, A_entry_max+1), repeat=4):
        if a*d - b*c == 1:
            mat = matrix(ZZ, [[a, b], [c, d]])
            r1 = eta_char_simple_exp(mat, n, True)
            r2 = eta_char_simple_exp(mat, n, False)
            assert(abs(sage.all.n(r1 - r2)) < 0.01)
            print(f'(a, b, c, d) = ({a}, {b}, {c}, {d}): {sage.all.n(r1)},{sage.all.n(r2)}')
    

def eta_char_gamma0N(N, es, *A):
    '''
    Calculate the logarithm char. of prod_{n mid N}eta^es[n]_(n tau) at some matrix.
    The meaning of the argument can be deduced from first four lines of the function body.
    '''
    if len(A) == 1: B = matrix(A[0])
    elif len(A) == 4: B = matrix(ZZ, [[A[0], A[1]], [A[2], A[3]]])
    elif len(A) == 2: B = matrix(ZZ, [A[0], A[1]])
    else:
        raise TypeError('The first argument should be a matrix,'
                        ' or a sequence of four numbers, or...')
    return sum(eta_char_simple(B, n) * es.get(n, 0) for n in N.divisors())


def eta_char_gamma0N_exp(N, es, *A):
    '''
    Calculate the char. of prod_{n mid N}eta^es[n]_(n\tau) at some matrix.
    Just call eta_char_gamma0N.
    Correspond to eq. (16) in the paper
    '''
    return exp(2 * sage.all.pi * sage.all.I * eta_char_gamma0N(N, es, *A))


def eta_order_gamma0N(N, es, cusp):
    '''
    Calculate the order of an eta quotient given by es at cusp.
    The cusp argument should be some element in QQ.
    To calculate order at infty, set cusp=1/N.
    '''
    d = cusp.denominator()
    return sum(gcd(n, d)^2 / n * es.get(n, 0) for n in N.divisors()) / 24


R.<q> = PowerSeriesRing(QQ)
_eta_power_cache = 0
def eta_power_series(precision=100):
    '''
    Compute the power series expansion of the Dedekind eta function
    excluding the q^(1/24) part.
    Use _eta_power_cache as cache.
    '''
    global _eta_power_cache
    if _eta_power_cache == 0 or _eta_power_cache.prec() < precision: 
        if _debug_mode:
            t0 = time.time()
            print('In eta_power_series(): recompute begins, need prec', precision)
        result = R(0)
        v_max = floor(sqrt(24 * precision + 1))
        for v in srange(1, v_max+1):
            if v % 12 == 1 or v % 12 == 11:
                result += q^((v^2 - 1) / 24)
            elif v % 12 == 5 or v % 12 == 7:
                result += -q^((v^2 - 1) / 24)
    
        result += O(q^precision)
        _eta_power_cache = result
        if _debug_mode:
            print('In eta_power_series(): recompute and cache edns, taking',
                  time.time()-t0, 'seconds.')
        
    return _eta_power_cache + O(q^precision)


def save_eta_power_cache(filename):
    '''
    Save _eta_power_cache to a file.
    Every time this program runs, one should call load_eta_power_cache(filename) if
    save_eta_power_cache(filename) has been called before. This will save much time
    since reproducing the eta_power_series costs much time.
    If one calls eta_power_series(precision) with precision greater than the cached one,
    then it automatically re-calculate the more precise eta power series.
    '''
    global _eta_power_cache
    with shelve.open(filename) as f:
        p = _eta_power_cache.prec()
        f['precision'] = p
        f['eta_power'] = _eta_power_cache
        
        
def load_eta_power_cache(filename, precision):
    '''
    Load from a file _eta_power_cache.
    '''
    global _eta_power_cache
    with shelve.open(filename) as f:
        p = f['precision']
        if _eta_power_cache == 0 or p > _eta_power_cache.prec():
            _eta_power_cache = f['eta_power'] + O(q^precision)
            

_eta_factor_cache = {}
def eta_factor(ex, m, precision=100):
    '''
    Return the power series of q^(-m*ex/24)*eta(m tau)^ex.
    The global variable _eta_factor_cache is a Python dict used for caching where
    each key is a tuple (ex, m) and corresponding value is the cached series.
    '''
    global _eta_factor_cache
    f = _eta_factor_cache.get((ex, m))
    if f == None or f.prec() < precision:
        if _debug_mode:
            t0 = time.time()
            print('In eta_factor(): recompute begins', (ex, m), precision)
            
        if ex != 1: s = eta_power_series(precision) ^ ex
        else: s = eta_power_series(precision)
            
        if _debug_mode:
            t1 = time.time()
            print('Taking', t1-t0, 'seconds.',
                  'In eta_factor(): substitute and caching begins')
             
        if m != 1: _eta_factor_cache[(ex, m)] = s.substitute({q: q^m})
        else: _eta_factor_cache[(ex, m)] = s
            
        if _debug_mode:
            print('In eta_factor(): substitute and caching ends, taking',
                  time.time()-t1, 'seconds')
    return _eta_factor_cache[(ex, m)] + O(q^precision)


def eta_factor_first_exp(ex, m):
    '''
    Return the exponent of the first non-zero q-term of eta(m\tau)^ex.
    '''
    return (ex * m) / 24


def eta_gamma_N(N, es, precision=100):
    '''
    Return certain eta-quotient on Gamma_0(N), as a pair whose first component
    is the exponent of the first non-zero power of q, and second component
    is the corresponding power series on q.
    The argument es is a Python dict each pair (k, v) in which means
    a factor eta(k\tau)^v.
    The returned object is the main object studied in the paper.
    '''
    N_divs = N.divisors()
    re = prod(eta_factor(es.get(n, 0), n, precision) for n in N_divs)
    ex = sum(eta_factor_first_exp(es.get(n, 0), n) for n in N_divs)
    return ex, re


def eta_gamma_N_series(N, es, precision=100):
    '''
    Just call eta_gamma_N(N, es, precision)[1].
    It returns the power series part of an eta-quotients,
    dropping the fractional-power part.
    '''
    return eta_gamma_N(N, es, precision)[1]


#For increasing the speed of p_num function, one should call
#p_num_init(m, n_max) first.
def p_num_init(m, n_max):
    return eta_factor(m, 1, n_max + 1)


@functools.lru_cache()
def p_num(m, n):
    '''
    Return the n-th coefficient of q^(-m/24)eta^m(tau).
    If n is not a non-negative integer, then return 0.
    '''
    if n not in ZZ or n < 0:
        return 0
    f = eta_factor(m, 1, n + 1)
    return f[n]


def coeff_eta_quotient(N, es, n):
    '''
    Return the n-th coefficient of the power series part of 
    the eta-quotient given by the Python dict es of level N.
    For example, a call of coeff_eta_quotient(4, {1: 3, 2: -2, 4: 10}, 17)
    returns the 17-th coefficient of the q-series of
    q^(39/24)eta^3(tau)eta^-2(2tau)eta^10(4tau).
    If n is not a non-negative integer, then return 0.
    '''
    if n not in ZZ or n < 0:
        return 0
    f = eta_gamma_N(N, es, precision=n+1)[1]
    return f[n]


@functools.lru_cache()
def some_eta_quotient_coef(r, p, y, m, beta=1):
    '''
    Return a_m^{r,p^\beta,y} in eq. (82) where m should be a non-negative integer.
    '''
    R.<q> = PowerSeriesRing(QQ)
    if beta % 2 == 1:
        exp_dict = {1: -24/gcd(12, p-1)*y, p: r + 24/gcd(12, p-1)*y}
    else:
        exp_dict = {1: r - 24/gcd(12, p-1)*y, p: 24/gcd(12, p-1)*y}
    _, f = eta_gamma_N(p, exp_dict, m + 1)
    return f[m]


def check_some_eta_quot_coef_recur(r, p, y, m, beta=1):
    '''
    This function is used to check Lemma 6.3 for a_m^{r,p^\beta,y}.
    '''
    a = []
    for n in srange(m + 1):
        a.append(some_eta_quotient_coef(r, p, y, n, beta))
    
    left = a[m]
    right = 0
    for k in srange(1, m + 1):
        t = 0
        if beta % 2 == 0:
            t += r * sigma(k)
        elif p.divides(k):
            t += r * p * sigma(k / p)
        if p.divides(k):
            t += 24 / gcd(12, p-1) * y * p * sigma(k / p)
        t -= 24 / gcd(12, p-1) * y * sigma(k)
        right += t * a[m - k]
    right = -right / m
    return left == right


def find_least_nonzero_pr(r, p, beta=1):
    '''
    Find the smallest integer y as described in Theorem 6.5.
    '''
    #p >= 5, r in Z,  or p = 2, 8|r,  or p = 3, 3|r
    if not p.is_prime() or not r in ZZ:
        raise ValueError('p should be a prime and r should be an integer!')
    if p == 2 and r % 8 != 0:
        raise ValueError('if p == 2, then r should be divisible by 8')
    if p == 3 and r % 3 != 0:
        raise ValueError('if p == 3, then r should be divisible by 3')
    if beta not in ZZ or beta < 1:
        raise ValueError('beta should be a positive integer')
    
    n0 = -floor((p^(2*floor(beta/2 + 1/2))-1)*r/(24*p^beta))
    n = n0
    while True:
        if p_num(r, (p^(2*floor(beta/2 + 1/2))-1)*r/24 + p^beta*n) != 0:
            return n
        n += 1
        
        
def beta_bar(beta):
    '''
    If beta is odd, then return 1,
    elif beta is even, then return 0.
    See the first paragraph of Section 6.
    '''
    if beta % 2 == 0:
        return 0
    elif beta % 2 == 1:
        return 1
    

@functools.lru_cache()
def cacul_cy(r, p, beta=1, f_cache=True):
    '''
    Return y_0, y_1 and the sequence c_y in Theorem 6.5.
    The sequence c_y is obtained by eq. (88).
    The sequence c_y is represented by a Python dict.
    The returned value is a tuple(cy, y0, y1) where cy is the dict.
    If \pi_p=(p-1)/gcd(12,p-1) does not divides the smallest y as described in Theorem 6.5,
    then a tuple (cy, None, None) is returned, in which case Theorem 6.5
    is not applicable.
    '''
    #p >= 5, r in Z,  or p = 2, 8|r,  or p = 3, 3|r
    re_dict = {}
    g, pb, pbp, pbm = gcd(12, p-1), p^beta, p^(beta+1), p^(beta-1)
    if r >= 0:
        y1_max_temp = -floor(-r*g/24*((p^(beta-beta_bar(beta))-1)/(pbm*(p-1))))
    else:
        y1_max_temp = -floor(r*g/24*((pbp - p^(1-beta_bar(beta)))/(p-1)))
    
    if f_cache:
        if _debug_mode:
            print('Test: p_num_init starts, y1_max_temp={}, r={}, m={}'.format(y1_max_temp,
                    r, (p^(2*floor((beta+1)/2))-1)*r/24 + pb*(p-1) / g * max([5, y1_max_temp]) + 1))
        p_num_init(r,
                   (p^(2*floor((beta+1)/2))-1)*r/24 + pb*(p-1) / g * max([5, y1_max_temp]) + 1)   
    if _debug_mode: print('Test: find_least_nonzero_pr starts')
    n0 = find_least_nonzero_pr(r, p, beta)
    if not Integer((p-1) / g).divides(n0):
        return re_dict, None, None
    
    y0 = n0 / (p-1) * g
    y1 = max([y0, y1_max_temp])
    
    #if f_cache=True, then the following call of some_eta_quotient_coef may
    #establish the cache and increase the speed of subsequent calls.
    if f_cache:
        if _debug_mode: print('Test: some_eta_quotient_coef init starts')
        for y in srange(y0, y1):
            seqc = some_eta_quotient_coef(r, p, y, (p-1) / g * (y1 - y), beta)
    for y in srange(y0, y1+1):
        if _debug_mode: print('Test: y0={}, y1={}, now proceeds {}...'.format(y0, y1, y))
        cy = p_num(r, (p^(2*floor((beta+1)/2))-1)*r/24 + pb*(p-1) / g * y) - \
            sum(re_dict[yp] * some_eta_quotient_coef(r, p, yp, (p-1) / g * (y - yp), beta)
                for yp in srange(y0, y))
        re_dict[y] = cy
    
    if _debug_mode: print('cacul_cy ends')
    return re_dict, y0, y1


@functools.lru_cache()
def check_cy(r, p, beta=1, f_cache=True, y0y1=False):
    '''
    This function is used to check the criterion eq. (90)
    If y0y1=True, then the returned value is a tuple (b, cy, y0, y1) where (cy, y0, y1) is the same
    as cacul_cy returns while b is a boolean value indicating the result of the checking process.
    If y0y1=False, the returned value is a tuple (b, cy).
    '''
    #p >= 5, r in Z,  or p = 2, 8|r,  or p = 3, 3|r
    cy_dict, y0, y1 = cacul_cy(r, p, beta, f_cache)
    if len(cy_dict) == 0: return (False, cy_dict, y0, y1) if y0y1 else (False, cy_dict)
    
    for m in srange((p-1)/gcd(12, p-1)*(y1-y0) + 1):
        left = p_num(r,
                (p^(2*floor((beta+1)/2))-1)*r/24 + p^beta*(p-1)/gcd(12, p-1)*y0 + p^beta*m)
        right = sum(cy_dict[y] * some_eta_quotient_coef(r, p, y, m - (p-1)/gcd(12, p-1)*(y-y0), beta)
                    for y in srange(y0, y0 + floor(gcd(12,p-1)/(p-1) * m) + 1))
        if left != right: return (False, cy_dict, y0, y1) if y0y1 else (False, cy_dict)
                    
    if y0y1 == False: return True, cy_dict
    else: return True, cy_dict, y0, y1
    

def show_identity(r, p, beta=1, f_cache=False, flag=0, fac=False):
    '''
    Return the formula (89) in Theorem 6.5 where all c_y are calculated explicitly.
    A returned value None means check_cy fails, i.e., eq. (90) does not hold.
    '''
    _, cy_dict, y0, y1 = check_cy(r, p, beta, f_cache, True)
    if _ == False: return None
    
    s1_f_left = 'sum_(m in {} + N) p_({})({}^{} m - ({}/24))q^m'
    t = [(p*r/24 if beta % 2 == 1 else r/24) + (p-1) / gcd(12, p-1) * y0,
        r,
        p,
        beta,
        r,
        ]
    s1_left = s1_f_left.format(*t)
    
    s2_f_left = 'q^({}) * sum_(m in N) p_({})({}^{} m + {})q^m'
    t2 = [p^beta_bar(beta) * r / 24 + (p-1)/gcd(12, p-1)*y0,
          r,
         p,
         beta,
         (p^(beta_bar(beta)+beta) - 1) * r / 24 + p^beta*(p-1)/gcd(12, p-1)*y0]
    s2_left = s2_f_left.format(*t2)
    
    s1_right = ''
    s2_right = ''
    for y in range(y0, y1+1):
        s1_right += '{}'.format(
            cy_dict[y] if not fac else (factor(cy_dict[y]) if cy_dict[y] != 0 else 0)) +\
        'eta^{}({}z)'.format(r, p if beta % 2 == 1 else '') +\
        '(eta({}z)eta^-1(z))^{}'.format(p, 24/gcd(12, p-1)*y)
        if y != y1: s1_right += ' + '
    
    for y in range(y0, y1+1):
        s2_right += '{}'.format(
            cy_dict[y] if not fac else (factor(cy_dict[y]) if cy_dict[y] != 0 else 0)) +\
        'eta^{}(z)'.format(r - 24/gcd(12,p-1)*y if beta % 2 == 0 else - 24/gcd(12,p-1)*y) +\
        'eta^{}({}z)'.format(r + 24/gcd(12,p-1)*y if beta % 2 == 1 else 24/gcd(12,p-1)*y, p)
        if y != y1: s2_right += ' + '
    if flag == 0:
        print(s1_left + ' = ' + s1_right)
    else:
        print(s2_left + ' = ' + s2_right)
        
        
def show_identity_latex(r, p, beta=1, f_cache=False, flag=0, fac=False):
    '''
    The same as show_identity with the returned formula in latex form.
    '''
    _, cy_dict, y0, y1 = check_cy(r, p, beta, f_cache, True)
    if _ == False: return None
    
    s1_f_left = '\\sum_{{m \\in {} + \\numgeq{{Z}}{{0}}}} P_{{{}}}\\left({}^{}m - \\frac{{{}}}{{24}}\\right)q^m'
    t = [(p*r/24 if beta % 2 == 1 else r/24) + (p-1) / gcd(12, p-1) * y0,
        r,
        p,
        beta,
        r,
        ]
    s1_left = s1_f_left.format(*t)
    
    s2_f_left = 'q^{{{}}}\\sum_{{n \\in \\numgeq{{Z}}{{0}}}} P_{{{}}}\\left({}^{}n + {}\\right)q^n'
    t2 = [p^beta_bar(beta) * r / 24 + (p-1)/gcd(12, p-1)*y0,
          r,
         p,
         beta,
         (p^(beta_bar(beta)+beta) - 1) * r / 24 + p^beta*(p-1)/gcd(12, p-1)*y0]
    s2_left = s2_f_left.format(*t2)
    
    s1_right = ''
    s2_right = ''
    for y in range(y0, y1+1):
        s1_right += '{}'.format(
            cy_dict[y] if not fac else (factor(cy_dict[y]) if cy_dict[y] != 0 else 0)) +\
        '\\eta^{{{}}}({}\\tau)'.format(r, p if beta % 2 == 1 else '') +\
        '(\\eta({}\\tau)eta^{{-1}}(\\tau))^{{{}}}'.format(p, 24/gcd(12, p-1)*y)
        if y != y1: s1_right += ' + '
    
    for y in range(y0, y1+1):
        s2_right += '{}'.format(
            cy_dict[y] if not fac else (latex(factor(cy_dict[y])) if cy_dict[y] != 0 else 0)) +\
        '\\eta^{{{}}}(\\tau)'.format(r - 24/gcd(12,p-1)*y if beta % 2 == 0 else - 24/gcd(12,p-1)*y) +\
        '\\eta^{{{}}}({}\\tau)'.format(r + 24/gcd(12,p-1)*y if beta % 2 == 1 else 24/gcd(12,p-1)*y, p)
        if y != y1: s2_right += ' + '
    if flag == 0:
        print(s1_left + ' = ' + s1_right)
    else:
        print(s2_left + ' = ' + s2_right)


def etapower_char_quotient(l, n, rn, rnp, A):
    '''
    An auxiliary function used to verify a formula of
    v_eta^rnp(a, bnl, c/nl, d)/v_eta^rn(a, bn, c/n, d).
    l, n should be positive integers
    rn, rnp integers
    A should be a matrix in Gamma0(nl)
    See Lemma 3.6.
    '''
    ((a, b), (c, d)) = (A[0], A[1])
    if c/n % 2 == 0:
        result = kronecker(c*n, d)^(rnp-rn)
        result *= kronecker(l, d)^rnp
        expnum = (a - 2*d) * c/n/l * (rnp - l*rn)
        expnum += -b*d*c^2/n/l * (rnp - l*rn)
        expnum += b*d*n * (l*rnp - rn)
        expnum += 3 * (d-1) * (rnp-rn)
        result *= exp(2*pi*I * expnum / 24)
    else:
        result = kronecker(d, c*n)^(rnp-rn)
        result *= kronecker(d, l)^rnp
        expnum = (a + d - 3) * c/n/l * (rnp - l*rn)
        expnum += -b*d*c^2/n/l * (rnp - l*rn)
        expnum += b*d*n * (l*rnp - rn)
        result *= exp(2*pi*I * expnum / 24)
    return result


def check_etapower_char_quotient(l, n, rn, rnp, A):
    '''Use etapower_char_quotient to compute a char. quotient
    and compare the result to that given by eta_char_gamma0N function
    the arguments are the same as etapower_char_quotient function
    '''
    re1 = etapower_char_quotient(l, n, rn, rnp, A)
    
    ((a, b), (c, d)) = (A[0], A[1])
    A_left = matrix(ZZ, [[a, b*n*l], [c/n/l, d]])
    A_right = matrix(ZZ, [[a, b*n], [c/n, d]])
    re2 = eta_char_gamma0N(1, {1: rnp}, A_left) - eta_char_gamma0N(1, {1: rn}, A_right)
    re2 = exp(2*pi*I * re2)
    
    if _debug_mode:
        print(sage.all.N(re1), sage.all.N(re2))
    return sage.all.N(re1 - re2)


def make_square(n):
    '''
    n should be a positive integer.
    Return the least positive square with the same prime factors as n
    '''
    assert n in ZZ and n > 0
    result = 1
    for key, value in dict(factor(n)).items():
        if value % 2 == 1:
            result *= key^(value+1)
        else:
            result *= key^value
    return result


@functools.lru_cache()
def gen_etaquot_dimone(b_cusp=False):
    '''Return a list containing all pairs (N, k) such that k > 0 and
    [SL(Z): Gamma0(N)]k < 12, so dim M_k(N, v) <= 1
    If b_cusp=True, then find those such that k > 0 and [SL(Z): Gamma0(N)]k = 12
    Table 1 and Table 5 in the paper are made according to the returned value of this function.
    '''
    re = []
    for N in srange(1, 25):
        for k in [r/2 for r in srange(1, 25)]:
            if not b_cusp and N * prod([1 + 1/p for p in N.prime_divisors()]) * k < 12:
                re.append((N, k))
            if b_cusp and N * prod([1 + 1/p for p in N.prime_divisors()]) * k == 12:
                re.append((N, k))
    return re


@functools.lru_cache()
def gen_holo_etaquot(N, k, nodict=False, b_cusp=False):
    '''Generate all holomorphic eta-quotients on Gamma0(N) of weight k or 
    holo. eta-quotients that are also cusp forms if b_cusp=True
    It can only deal with (N, k) in the list returned by gen_etaquot_dimone()
    or gen_etaquot_dimone(True) if b_cusp=True
    since in general cases I don't know the bound of x_n
    Table 2 and Table 3 in the paper are made according to the returned value of this function.
    '''
    assert (N, k) in gen_etaquot_dimone(b_cusp)
    re = []
    N_divs = N.divisors()
    list_es = itertools.product(srange(-20, 30), repeat=len(N_divs))
    for es in list_es:
        if sum(es) != 2*k: continue
        es_dict = {N_divs[j]: es[j] for j in range(len(N_divs))}
        if not b_cusp:
            for n in N_divs:
                if eta_order_gamma0N(N, es_dict, 1/n) < 0:
                    break
            else:
                if not nodict:
                    re.append(es_dict)
                else:
                    re.append(tuple([(N_divs[j], es[j]) for j in range(len(N_divs))]))
        else:
            for n in N_divs:
                if eta_order_gamma0N(N, es_dict, 1/n) <= 0:
                    break
            else:
                if not nodict:
                    re.append(es_dict)
                else:
                    re.append(tuple([(N_divs[j], es[j]) for j in range(len(N_divs))]))
    return re
        

def findl_intwt(N, k, omitempty=False, b_cusp=False):
    '''
    Given level N and integral weight k, this function returns a Python dict where
    each item is of the form (eta^r, eta^rp) : ls, where ls is a list containing integers l
    from 1 to 24 such that v_r and v_rp is l-compatible.
    Theorem 4.2 is proved and Table A.6 is generated according to the returned value of this function
    '''
    assert k in ZZ
    N_divs = N.divisors()
    re = {}
    holo_etaquots = gen_holo_etaquot(N, k, nodict=True, b_cusp=b_cusp)
    for etaquot_left in holo_etaquots:
        for etaquot_right in holo_etaquots:
            etaquot_left_d = {n: rn for n, rn in etaquot_left}
            etaquot_right_d = {n: rn for n, rn in etaquot_right}
            ls = []
            if _debug_mode:
                print(etaquot_left, etaquot_right, etaquot_left_d, etaquot_right_d)
            delta = prod(n for n in N_divs if (etaquot_left_d[n] - etaquot_right_d[n]) % 2 == 1)
            if not delta.is_square():
                if not omitempty:
                    re[(etaquot_left, etaquot_right)] = ls
                continue
            value1_le = sum([N/n * etaquot_left_d[n] for n in N_divs])
            value1_ri = sum([N/n * etaquot_right_d[n] for n in N_divs])
            value2_le = sum([n * etaquot_left_d[n] for n in N_divs])
            value2_ri = sum([n * etaquot_right_d[n] for n in N_divs])
            for l in srange(1, 25):
                if (l*value1_le - value1_ri) % 24 != 0: continue
                if (value2_le - l*value2_ri) % 24 != 0: continue
                ls.append(l)
            if not omitempty or len(ls) != 0:
                re[(etaquot_left, etaquot_right)] = ls
    return re


def findl_halfintwt(N, k, omitempty=False, b_cusp=False):
    '''
    Given level N and half-integral weight k, this function returns a Python dict where
    each item is of the form (eta^r, eta^rp) : (l0, ms), where ms is a list containing integers m^2 mod 24
    from 0 to 23 such that v_r and v_rp is (l0*m^2)-compatible.
    Theorem 4.3 is proved and Table A.7 is generated according to the returned value of this function
    '''
    assert k - 1/2 in ZZ
    N_divs = N.divisors()
    re = {}
    holo_etaquots = gen_holo_etaquot(N, k, nodict=True, b_cusp=b_cusp)
    for etaquot_left in holo_etaquots:
        for etaquot_right in holo_etaquots:
            etaquot_left_d = {n: rn for n, rn in etaquot_left}
            etaquot_right_d = {n: rn for n, rn in etaquot_right}
            ms = []
            if _debug_mode:
                print(etaquot_left, etaquot_right, etaquot_left_d, etaquot_right_d)
            prodn = prod(n for n in N_divs if (etaquot_left_d[n] - etaquot_right_d[n]) % 2 == 1)
            l0 = make_square(prodn) / prodn
            value1_le = sum([N/n * etaquot_left_d[n] for n in N_divs])
            value1_ri = sum([N/n * etaquot_right_d[n] for n in N_divs])
            value2_le = sum([n * etaquot_left_d[n] for n in N_divs])
            value2_ri = sum([n * etaquot_right_d[n] for n in N_divs])
            for m in srange(1, 25):
                if (l0*m^2*value1_le - value1_ri) % 24 != 0: continue
                if (value2_le - l0*m^2*value2_ri) % 24 != 0: continue
                ms.append(m^2 % 24)
            if not omitempty or len(ms) != 0:
                re[(etaquot_left, etaquot_right)] = (l0, ms)
    return re


def findl(N, k, omitempty=False, b_cusp=False):
    '''
    This is a wrapped function of findl_intwt and findl_halfintwt.
    '''
    if k in ZZ:
        return findl_intwt(N, k, omitempty, b_cusp)
    elif k - 1/2 in ZZ:
        return findl_halfintwt(N, k, omitempty, b_cusp)
    
    
def check_compatible(N, dchar_left, echar_left, dchar_right, echar_right, l):
    '''
    To check the main theorem on compatibility of T_l (Theorem 3.9).
    dchar_left and dchar_right should be two Dirichlet characters,
    and echar_left, echar_right should be two Python dict indicating eta-quotients
    while N is the level. The parameter l represents the operator T_l
    '''
    grp = Gamma0(N * l)
    gens = grp.gens()
    for gen in gens:
        (a, b), (c, d) = gen[0], gen[1]
        lvalue = dchar_left(d) * eta_char_gamma0N_exp(N, echar_left, a, b, c, d)
        rvalue = dchar_right(d) * eta_char_gamma0N_exp(N, echar_right, a, b*l, c/l, d)
        if lvalue != rvalue:
            return False
        
    a, b, c, d = -1, 0, 0, -1
    lvalue = dchar_left(d) * eta_char_gamma0N_exp(N, echar_left, a, b, c, d)
    rvalue = dchar_right(d) * eta_char_gamma0N_exp(N, echar_right, a, b*l, c/l, d)
    if lvalue != rvalue:
        return False
    
    return True


def some_gauss_sum(m, t):
    '''
    This returns the Gauss sum in the LHS of eq. (60) in Lemma 5.6
    '''
    return sum(kronecker(b, m) * exp(n(2*pi*I*t*b/m)) for b in srange(m))


#The following Python functions correspond to functions in Def. 2 in the paper.
rad = radical
def irad(m):
    return ZZ(m / rad(m))
def radE(m):
    pps = dict(factor(m))
    return prod(key for key, value in pps.items() if value % 2 == 0)
def radO(m):
    pps = dict(factor(m))
    return prod(key for key, value in pps.items() if value % 2 == 1)
def radp(m):
    return rad(m) * radE(m)
def iradp(m):
    return ZZ(m / radp(m))


def test_rad_seq():
    '''
    This is used to check the above six functions by listing their values for m=1,2,...,299.
    '''
    results = []
    for m in srange(1, 300):
        results.append([factor(m), factor(rad(m)), factor(radE(m)),
                       factor(radO(m)), factor(radp(m)), factor(irad(m)), factor(iradp(m))])
        
    T = table(results, header_row=['$m$', '$rad(m)$', '$rad_E(m)$', '$rad_O(m)$',
                                  "$rad'(m)$", '$irad(m)$', "$irad'(m)$"])
    display(T)
    
    
def some_gauss_sum_formula_all(m, t):
    '''
    This returns the value of the expression in the RHS of eq. (60) in Lemma 5.6
    '''
    if not irad(m).divides(t):
        return 0
    
    u = len([p for p in radO(m).prime_divisors() if p % 4 == 3])
    re = I^(u^2)
    re *= m / sqrt(radp(m))
    re *= kronecker(t / iradp(m), radO(m))
    re *= prod((p-1-p*kronecker(t/irad(m), p)^2) for p in radE(m).prime_divisors())
    return re

    
def psi_rab_new(r, a, b, d):
    '''
    This quantity occurs in the Fourier expansion of T_l eta^r.
    See Lemma 5.2.
    '''
    
    if r % 2 == 0:
        return 1
    else:
        return kronecker(-b, gcd(a, d))
    

def coef_Tletar_a_new(l, r, n, a):
    '''
    Let l^(r/4)Tl eta^r(z) = sum_(n in Z) sum_(a | l) E a^(r/2)P_r((n/a^2-r)/24)q^(n/(24l))
    Then E can be calculated by calling coef_Tletar_a_new(l, r, n, a)
    In another words, this returns a factor in RHS of eq. (55).
    '''
    assert r in ZZ
    assert r*(l-1) % 24 == 0
    if r % 2 != 0:
        assert l.is_square()
        
    re = 0
    d = Integer(l/a)
    for b in srange(d):
        if gcd([a, b, d]) == 1:
            temp = (n - r*l^2) * b / (24*a*l)
            re += psi_rab_new(r, a, b, d) * exp(2*pi*I * temp)
    re *= exp(-2*pi*I * r*(d-1)/8)
    return re


def coef_Tletar_new(l, r, n):
    '''
    Let l^(r/4)Tl eta^r(z) = sum_(n in Z) F q^(n/24l)
    Then F can be calculated by calling coef_Tletar(l, r, n)
    In another words, this returns the coefficient of the q^(n/24l) term of eq. (55).
    This function calculates this quantity according to its original expression.
    '''
    assert r*(l-1) % 24 == 0
    if r % 2 != 0:
        assert l.is_square()
        
    re = 0
    for a in l.divisors():
        if p_num(r, (n/a^2 - r) / 24) != 0:
            re += coef_Tletar_a_new(l, r, n, a) * a^(r/2) * p_num(r, (n/a^2 - r) / 24)
    return re


@functools.lru_cache()
def coef_Tletar_good_formula(l, r, n):
    '''
    The same as coef_Tletar_new(l, r, n), but this function calculates
    using Lemma 5.5 (when 2 \mid r) or Lemma 5.7 (when 2 \nmid r).
    '''
    assert l > 0 and r*(l-1) % 24 == 0
    if r % 2 == 1:
        assert l.is_square()
    
    re = 0
    if r % 2 == 0:
        for a in l.divisors():
            d = Integer(l / a)
            if (a^2).divides(n) and (n/a^2 - r) % 24 == 0:
                temp = Integer(24*a*l / gcd(24*a*l, n-r*l^2))
                for t in gcd(a, d).divisors():
                    if temp.divides(t):
                        re += a^(r/2) * (-1)^(r*(d-1)/4) * moebius(t) * d/t *\
                        p_num(r, (n/a^2 - r) / 24)
    else:    # r % 2 == 1
        if n % l != 0 or (ZZ(n/l) - r) % 24 != 0:
            return 0
        # now we can assume that n % l = 0 and (ZZ(n/l) - r) % 24 == 0
        np = ZZ(n / l)
        for a in l.divisors():
            d = ZZ(l / a)
            if (24*a).divides(rad(gcd(a, d)) * (np-r*l)):
                Prnum = p_num(r, (n / a^2 - r) / 24)
                if Prnum == 0:
                    continue
                temp_e = 2 * (-1)^((r-1)/2)
                temp = kronecker(temp_e, d)
                temp *= kronecker(radE(gcd(a, d)), radO(gcd(a, d)))
                temp *= d * a^(r/2) / sqrt(radp(gcd(a, d)))
                temp *= Prnum
                temp *= kronecker(rad(gcd(a, d)) * (np-r*l) / (24*a), radO(gcd(a, d)))
                for p in radE(gcd(a, d)).prime_divisors():
                    temp *= (p-1-p*kronecker(rad(gcd(a, d)) * (np-r*l) / (24*a), p)^2)
                re += temp
    return re


def check_main_theorem_onTletar(l, r, n_min=-100, n_max=2400):
    '''
    Check Theorem 5.8 for n_min <= n <= n_max.
    '''
    assert l in ZZ and l > 0
    assert r in ZZ
    assert 24.divides(r*(l-1))
    if r % 2 == 1:
        assert l.is_square()
    
    for n in range(n_min, n_max + 1):
        n = Integer(n)
        left_h = coef_Tletar_good_formula(l, r, l*n)
        right_h = coef_Tletar_good_formula(l, r, l*r) * p_num(r, (n - r)/24)
        assert left_h == right_h
        if _debug_mode:
            print(f' n={n}, left={left_h}, right={right_h}')
    
    print(f'r={r}, l={l} OK\n')

<>:345: DeprecationWarning: invalid escape sequence \p
<>:817: DeprecationWarning: invalid escape sequence \m
<>:345: DeprecationWarning: invalid escape sequence \p
<>:817: DeprecationWarning: invalid escape sequence \m
<>:345: DeprecationWarning: invalid escape sequence \p
<>:817: DeprecationWarning: invalid escape sequence \m
<ipython-input-1-b09091b4186d>:345: DeprecationWarning: invalid escape sequence \p
  '''
<ipython-input-1-b09091b4186d>:817: DeprecationWarning: invalid escape sequence \m
  '''


# Examples of usage

In [2]:
eta_char_gamma0N_exp(4, {1: 2, 2: 3, 4: -5}, 1, 0, 4, 1)

-(1/2*I + 1/2)*sqrt(2)

In [3]:
eta_order_gamma0N(4, {1: 2, 2: 3, 4: -5}, 1/4)

-1/2

In [4]:
p_num(-1, 10000)

36167251325636293988820471890953695495016030339315650422081868605887952568754066420592310556052906916435144

In [5]:
coeff_eta_quotient(4, {1: 2, 2: 3, 4: -5}, 3)

8

In [6]:
eta_gamma_N(4, {1: 2, 2: 3, 4: -5}, 10)

(-1/2,
 1 - 2*q - 4*q^2 + 8*q^3 + 9*q^4 - 14*q^5 - 20*q^6 + 24*q^7 + 39*q^8 - 52*q^9 + O(q^10))

In [7]:
chi = DirichletGroup(4)[0]
v1 = {1: 2, 2: -3, 4: 5}
v2 = {1: 3, 2: 3, 4: -2}
for l in srange(1, 49):
    print(f'l={l}, compatible={check_compatible(4, chi, v1, chi, v2, l)}')

l=1, compatible=False
l=2, compatible=False
l=3, compatible=False
l=4, compatible=False
l=5, compatible=False
l=6, compatible=False
l=7, compatible=False
l=8, compatible=False
l=9, compatible=False
l=10, compatible=False
l=11, compatible=False
l=12, compatible=False
l=13, compatible=False
l=14, compatible=False
l=15, compatible=False
l=16, compatible=True
l=17, compatible=False
l=18, compatible=False
l=19, compatible=False
l=20, compatible=False
l=21, compatible=False
l=22, compatible=False
l=23, compatible=False
l=24, compatible=False
l=25, compatible=False
l=26, compatible=False
l=27, compatible=False
l=28, compatible=False
l=29, compatible=False
l=30, compatible=False
l=31, compatible=False
l=32, compatible=False
l=33, compatible=False
l=34, compatible=False
l=35, compatible=False
l=36, compatible=False
l=37, compatible=False
l=38, compatible=False
l=39, compatible=False
l=40, compatible=True
l=41, compatible=False
l=42, compatible=False
l=43, compatible=False
l=44, compatible=False

In [8]:
gen_etaquot_dimone(b_cusp=False)

[(1, 1/2),
 (1, 1),
 (1, 3/2),
 (1, 2),
 (1, 5/2),
 (1, 3),
 (1, 7/2),
 (1, 4),
 (1, 9/2),
 (1, 5),
 (1, 11/2),
 (1, 6),
 (1, 13/2),
 (1, 7),
 (1, 15/2),
 (1, 8),
 (1, 17/2),
 (1, 9),
 (1, 19/2),
 (1, 10),
 (1, 21/2),
 (1, 11),
 (1, 23/2),
 (2, 1/2),
 (2, 1),
 (2, 3/2),
 (2, 2),
 (2, 5/2),
 (2, 3),
 (2, 7/2),
 (3, 1/2),
 (3, 1),
 (3, 3/2),
 (3, 2),
 (3, 5/2),
 (4, 1/2),
 (4, 1),
 (4, 3/2),
 (5, 1/2),
 (5, 1),
 (5, 3/2),
 (6, 1/2),
 (7, 1/2),
 (7, 1),
 (8, 1/2),
 (9, 1/2),
 (10, 1/2),
 (11, 1/2),
 (13, 1/2),
 (17, 1/2),
 (19, 1/2)]

In [9]:
gen_holo_etaquot(N=4, k=3/2)

[{1: -6, 2: 15, 4: -6},
 {1: -5, 2: 12, 4: -4},
 {1: -5, 2: 13, 4: -5},
 {1: -4, 2: 9, 4: -2},
 {1: -4, 2: 10, 4: -3},
 {1: -4, 2: 11, 4: -4},
 {1: -4, 2: 12, 4: -5},
 {1: -3, 2: 6, 4: 0},
 {1: -3, 2: 7, 4: -1},
 {1: -3, 2: 8, 4: -2},
 {1: -3, 2: 9, 4: -3},
 {1: -3, 2: 10, 4: -4},
 {1: -2, 2: 3, 4: 2},
 {1: -2, 2: 4, 4: 1},
 {1: -2, 2: 5, 4: 0},
 {1: -2, 2: 6, 4: -1},
 {1: -2, 2: 7, 4: -2},
 {1: -2, 2: 8, 4: -3},
 {1: -2, 2: 9, 4: -4},
 {1: -1, 2: 0, 4: 4},
 {1: -1, 2: 1, 4: 3},
 {1: -1, 2: 2, 4: 2},
 {1: -1, 2: 3, 4: 1},
 {1: -1, 2: 4, 4: 0},
 {1: -1, 2: 5, 4: -1},
 {1: -1, 2: 6, 4: -2},
 {1: -1, 2: 7, 4: -3},
 {1: 0, 2: -3, 4: 6},
 {1: 0, 2: -2, 4: 5},
 {1: 0, 2: -1, 4: 4},
 {1: 0, 2: 0, 4: 3},
 {1: 0, 2: 1, 4: 2},
 {1: 0, 2: 2, 4: 1},
 {1: 0, 2: 3, 4: 0},
 {1: 0, 2: 4, 4: -1},
 {1: 0, 2: 5, 4: -2},
 {1: 0, 2: 6, 4: -3},
 {1: 1, 2: -3, 4: 5},
 {1: 1, 2: -2, 4: 4},
 {1: 1, 2: -1, 4: 3},
 {1: 1, 2: 0, 4: 2},
 {1: 1, 2: 1, 4: 1},
 {1: 1, 2: 2, 4: 0},
 {1: 1, 2: 3, 4: -1},
 {1: 1, 2: 4, 

In [10]:
findl(6, 1, omitempty=True, b_cusp=True)

{(((1, -2), (2, 3), (3, 3), (6, -2)), ((1, -2), (2, 3), (3, 3), (6, -2))): [1],
 (((1, -2), (2, 3), (3, 3), (6, -2)), ((1, 0), (2, 1), (3, 1), (6, 0))): [5],
 (((1, -2), (2, 3), (3, 3), (6, -2)), ((1, 1), (2, 0), (3, 0), (6, 1))): [7],
 (((1, -2), (2, 3), (3, 3), (6, -2)),
  ((1, 3), (2, -2), (3, -2), (6, 3))): [11],
 (((1, -2), (2, 4), (3, 1), (6, -1)), ((1, -2), (2, 4), (3, 1), (6, -1))): [1],
 (((1, -2), (2, 4), (3, 1), (6, -1)), ((1, -1), (2, 1), (3, 4), (6, -2))): [3],
 (((1, -2), (2, 4), (3, 1), (6, -1)), ((1, 0), (2, 0), (3, 1), (6, 1))): [3],
 (((1, -2), (2, 4), (3, 1), (6, -1)), ((1, 1), (2, -1), (3, -2), (6, 4))): [3],
 (((1, -2), (2, 4), (3, 1), (6, -1)), ((1, 1), (2, 1), (3, 0), (6, 0))): [9],
 (((1, -2), (2, 4), (3, 1), (6, -1)),
  ((1, 4), (2, -2), (3, -1), (6, 1))): [17],
 (((1, -1), (2, 1), (3, 2), (6, 0)), ((1, -1), (2, 1), (3, 2), (6, 0))): [1],
 (((1, -1), (2, 1), (3, 2), (6, 0)), ((1, 0), (2, 2), (3, 1), (6, -1))): [7],
 (((1, -1), (2, 1), (3, 2), (6, 0)), ((1, 1), 

In [11]:
for m in srange(1, 200, 2):
    print(f'm={m}', end=' ')
    for t in srange(m - 30, m + 30):
        assert numerical_approx(some_gauss_sum(m, t) - some_gauss_sum_formula_all(m, t)) <= 0.01
    print('ok.')

m=1 ok.
m=3 ok.
m=5 ok.
m=7 ok.
m=9 ok.
m=11 ok.
m=13 ok.
m=15 ok.
m=17 ok.
m=19 ok.
m=21 ok.
m=23 ok.
m=25 ok.
m=27 ok.
m=29 ok.
m=31 ok.
m=33 ok.
m=35 ok.
m=37 ok.
m=39 ok.
m=41 ok.
m=43 ok.
m=45 ok.
m=47 ok.
m=49 ok.
m=51 ok.
m=53 ok.
m=55 ok.
m=57 ok.
m=59 ok.
m=61 ok.
m=63 ok.
m=65 ok.
m=67 ok.
m=69 ok.
m=71 ok.
m=73 ok.
m=75 ok.
m=77 ok.
m=79 ok.
m=81 ok.
m=83 ok.
m=85 ok.
m=87 ok.
m=89 ok.
m=91 ok.
m=93 ok.
m=95 ok.
m=97 ok.
m=99 ok.
m=101 ok.
m=103 ok.
m=105 ok.
m=107 ok.
m=109 ok.
m=111 ok.
m=113 ok.
m=115 ok.
m=117 ok.
m=119 ok.
m=121 ok.
m=123 ok.
m=125 ok.
m=127 ok.
m=129 ok.
m=131 ok.
m=133 ok.
m=135 ok.
m=137 ok.
m=139 ok.
m=141 ok.
m=143 ok.
m=145 ok.
m=147 ok.
m=149 ok.
m=151 ok.
m=153 ok.
m=155 ok.
m=157 ok.
m=159 ok.
m=161 ok.
m=163 ok.
m=165 ok.
m=167 ok.
m=169 ok.
m=171 ok.
m=173 ok.
m=175 ok.
m=177 ok.
m=179 ok.
m=181 ok.
m=183 ok.
m=185 ok.
m=187 ok.
m=189 ok.
m=191 ok.
m=193 ok.
m=195 ok.
m=197 ok.
m=199 ok.


In [12]:
l = 9
r = 12
print(f'l={l}, r={r}')
for n in srange(0, 73):
    print(f'The coefficient of q^({n}/24)-term: {coef_Tletar_good_formula(l, r, l*n)} another way {coef_Tletar_new(l, r, l*n)}')

l=9, r=12
The coefficient of q^(0/24)-term: 0 another way 0
The coefficient of q^(1/24)-term: 0 another way 0
The coefficient of q^(2/24)-term: 0 another way 0
The coefficient of q^(3/24)-term: 0 another way 0
The coefficient of q^(4/24)-term: 0 another way 0
The coefficient of q^(5/24)-term: 0 another way 0
The coefficient of q^(6/24)-term: 0 another way 0
The coefficient of q^(7/24)-term: 0 another way 0
The coefficient of q^(8/24)-term: 0 another way 0
The coefficient of q^(9/24)-term: 0 another way 0
The coefficient of q^(10/24)-term: 0 another way 0
The coefficient of q^(11/24)-term: 0 another way 0
The coefficient of q^(12/24)-term: -1620 another way -1620
The coefficient of q^(13/24)-term: 0 another way 0
The coefficient of q^(14/24)-term: 0 another way 0
The coefficient of q^(15/24)-term: 0 another way 0
The coefficient of q^(16/24)-term: 0 another way 0
The coefficient of q^(17/24)-term: 0 another way 0
The coefficient of q^(18/24)-term: 0 another way 0
The coefficient of q^(1

In [13]:
check_main_theorem_onTletar(9, 3, n_min=-6000, n_max=6000)

r=3, l=9 OK



In [14]:
 check_some_eta_quot_coef_recur(13, 37, -8, 6, 3)

True

In [15]:
show_identity_latex(-1, 5, 3)

\sum_{m \in 19/24 + \numgeq{Z}{0}} P_{-1}\left(5^3m - \frac{-1}{24}\right)q^m = 169229875\eta^{-1}(5\tau)(\eta(5\tau)eta^{-1}(\tau))^{6} + 29453534562500\eta^{-1}(5\tau)(\eta(5\tau)eta^{-1}(\tau))^{12} + 261225460367187500\eta^{-1}(5\tau)(\eta(5\tau)eta^{-1}(\tau))^{18} + 530364966230302734375\eta^{-1}(5\tau)(\eta(5\tau)eta^{-1}(\tau))^{24} + 422572061722889404296875\eta^{-1}(5\tau)(\eta(5\tau)eta^{-1}(\tau))^{30} + 172235628573713317871093750\eta^{-1}(5\tau)(\eta(5\tau)eta^{-1}(\tau))^{36} + 41826649037017536163330078125\eta^{-1}(5\tau)(\eta(5\tau)eta^{-1}(\tau))^{42} + 6664657315766966342926025390625\eta^{-1}(5\tau)(\eta(5\tau)eta^{-1}(\tau))^{48} + 743575088309750854969024658203125\eta^{-1}(5\tau)(\eta(5\tau)eta^{-1}(\tau))^{54} + 60806885429807044565677642822265625\eta^{-1}(5\tau)(\eta(5\tau)eta^{-1}(\tau))^{60} + 3767317361076713539659976959228515625\eta^{-1}(5\tau)(\eta(5\tau)eta^{-1}(\tau))^{66} + 181185390672895591706037521362304687500\eta^{-1}(5\tau)(\eta(5\tau)eta^{-1}(\tau))